In [25]:
import numpy as np
from six.moves import cPickle as pickle
import tensorflow as tf

pixels_height = 32
pixels_width = 32
num_labels = 11
num_channels = 1 #grayscale

In [26]:
def getDataAndLabel(dataset):
    return dataset['data'], dataset['label']
pickleFile = open('tensorflow_data.pickle', 'r')
tensorflowData = pickle.load(pickleFile)
train_X, train_labels = getDataAndLabel(tensorflowData['train'])
test_X, test_labels = getDataAndLabel(tensorflowData['test'])
valid_X, valid_labels = getDataAndLabel(tensorflowData['valid'])


In [28]:
print(len(train_X))
print(len(test_X))
print(len(valid_X))

33402
13068
10000


In [29]:
def reformatData(X):
    return X.reshape(-1, pixels_width, pixels_height, num_channels).astype(np.float32)
   

train_dataset = reformatData(train_X)
valid_dataset = reformatData(valid_X)
test_dataset = reformatData(test_X)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (33402, 32, 32, 1), (33402, 6))
('Validation set', (10000, 32, 32, 1), (10000, 6))
('Test set', (13068, 32, 32, 1), (13068, 6))


In [30]:
batch_size = 64
patch_size = 5

depth_1 = 32
depth_2 = 64
num_hidden = 64
shape = [batch_size, pixels_width, pixels_height, num_channels]


In [32]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [33]:
graph = tf.Graph()
with graph.as_default():
    # Input data
    # Load training, testing and validation data into constants that
    # are attached to the graph
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, pixels_width, pixels_height, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Conv1
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_1], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth_1]))
    
    #Conv2
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, pixels_width, depth_2], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth_2]), dtype=tf.float32)
    
    #Fully connected 1
    layer3_weights = tf.Variable(tf.truncated_normal([pixels_width // 4 * pixels_height // 4 * 64, 512], stddev=0.1, dtype=tf.float32))
    layer3_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=tf.float32))
    
    #Fully connected 2
    layer4_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    layer4_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
        
        
    s1_w = tf.get_variable("WS1", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s1_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS1')
    s2_w = tf.get_variable("WS2", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s2_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS2')
    """
    s3_w = tf.get_variable("WS3", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s3_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS3')
    s4_w = tf.get_variable("WS4", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s4_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS4')
    s5_w = tf.get_variable("WS5", shape=[num_hidden1, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s5_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS5')"""
    
    #model
    def model(data, isTrain = False):
        conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 1, 1, 1], padding='SAME')
        
        relu = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding ='SAME')
        
        conv = tf.nn.conv2d(pool, layer2_weights, strides = [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        if isTrain == True:
            hidden = tf.nn.dropout(hidden, 0.5)
        
        logits = tf.matmul(hidden, layer4_weights) + layer4_biases
        return [logits, logits, logits, logits, logits]
    
    logits = model(tf_train_dataset)
    # Training computation.
    
    # Define loss function.
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[0], tf_train_labels[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[1], tf_train_labels[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[2], tf_train_labels[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[3], tf_train_labels[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits[4], tf_train_labels[:,5]))
    
    # Optimizer.
    global_step = tf.Variable(0)
    #learning rate with exponential decay.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.95) 
    optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.pack([tf.nn.softmax(model(tf_train_dataset, True)[0]),\
                      tf.nn.softmax(model(tf_train_dataset, True)[1]),\
                      tf.nn.softmax(model(tf_train_dataset, True)[2]),\
                      tf.nn.softmax(model(tf_train_dataset, True)[3]),\
                      tf.nn.softmax(model(tf_train_dataset, True)[4])])
    valid_prediction = tf.pack([tf.nn.softmax(model(tf_valid_dataset)[0]),\
                      tf.nn.softmax(model(tf_valid_dataset)[1]),\
                      tf.nn.softmax(model(tf_valid_dataset)[2]),\
                      tf.nn.softmax(model(tf_valid_dataset)[3]),\
                      tf.nn.softmax(model(tf_valid_dataset)[4])])
    test_prediction = tf.pack([tf.nn.softmax(model(tf_test_dataset)[0]),\
                     tf.nn.softmax(model(tf_test_dataset)[1]),\
                     tf.nn.softmax(model(tf_test_dataset)[2]),\
                     tf.nn.softmax(model(tf_test_dataset)[3]),\
                     tf.nn.softmax(model(tf_test_dataset)[4])])
    saver = tf.train.Saver()

In [34]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()

  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0): 
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels[:,1:6]))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels[:,1:6]))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))
  save_path = saver.save(session, "SVHN_MODEL.ckpt")
  print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 20.592758
Minibatch accuracy: 3.4%
Validation accuracy: 67.3%
Minibatch loss at step 500: 4.971775
Minibatch accuracy: 78.8%
Validation accuracy: 67.3%
Minibatch loss at step 1000: 5.721244
Minibatch accuracy: 74.4%
Validation accuracy: 67.3%
Minibatch loss at step 1500: 5.284844
Minibatch accuracy: 77.2%
Validation accuracy: 67.3%
Minibatch loss at step 2000: 5.749935
Minibatch accuracy: 74.1%
Validation accuracy: 67.3%
Minibatch loss at step 2500: 5.282592
Minibatch accuracy: 76.2%
Validation accuracy: 67.3%
Minibatch loss at step 3000: 5.306844
Minibatch accuracy: 76.6%
Validation accuracy: 67.3%
Minibatch loss at step 3500: 5.722241
Minibatch accuracy: 73.8%
Validation accuracy: 67.3%
Minibatch loss at step 4000: 5.392472
Minibatch accuracy: 75.6%
Validation accuracy: 67.3%
Minibatch loss at step 4500: 5.541424
Minibatch accuracy: 75.0%
Validation accuracy: 67.3%
Minibatch loss at step 5000: 5.766647
Minibatch accuracy: 73.8%
Validation accurac